**What is going modular?**

Going modular involves turning notebook code (from a Jupyter Notebook or Google Colab notebook) into a series of different Python scripts that offer similar functionality.

For example, we could turn our notebook code from a series of cells into the following Python files:

1/ `data_setup.py` - a file to prepare and download data if needed.

2/ `engine.py` - a file containing various training functions.

3/ `model_builder.py` or `model.py` - a file to create a PyTorch model.

4/ `train.py` - a file to leverage all other files and train a target PyTorch model.

5/ `utils.py` - a file dedicated to helpful utility functions.

>**Note:** The naming and layout of the above files will depend on your use case and code requirements. Python scripts are as general as individual notebook cells, meaning, you could create one for almost any kind of functionality.

**Example**

For example, you might be instructed to run code like the following in a terminal/command line to train a model:

`python train.py --model MODEL_NAME --batch_size BATCH_SIZE --lr LEARNING_RATE --num_epochs NUM_EPOCHS`

In this case, train.py is the target Python script, it'll likely contain functions to train a PyTorch model.

And --model, --batch_size, --lr and --num_epochs are known as **argument flags**.

You can set these to whatever values you like and if they're compatible with train.py, they'll work, if not, they'll error.

For example, let's say we wanted to train our **TinyVGG model** from notebook 04 for **10 epochs** with a **batch size of 32** and a **learning rate of 0.001**:

`python train.py --model tinyvgg --batch_size 32 --lr 0.001 --num_epochs 10`


In this session, we want to create our directiory like this:

    data/
      pizza_steak_sushi/
        train/
          pizza/
            train_image_01.jpeg
            train_image_02.jpeg
            ...
          steak/
          sushi/
        test/
          pizza/
            test_image_01.jpeg
            test_image_02.jpeg
            ...
          steak/
          sushi/
    going_modular/
      data_setup.py
      engine.py
      model_builder.py
      train.py
      utils.py
    models/
      saved_model.pth

### **1/ Get Data**

In [9]:
import os
import requests
import zipfile
from pathlib import Path

# Setup path to data folder
data_path = Path("data/")
image_path = data_path / "pizza_steak_sushi"

# If the image folder doesn't exist, download it and prepare it...
if image_path.is_dir():
    print(f"{image_path} directory exists.")
else:
    print(f"Did not find {image_path} directory, creating one...")
    image_path.mkdir(parents=True, exist_ok=True)

# Download pizza, steak, sushi data
with open(data_path / "pizza_steak_sushi.zip", "wb") as f:
    request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
    print("Downloading pizza, steak, sushi data...")
    f.write(request.content)

# Unzip pizza, steak, sushi data
with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_ref:
    print("Unzipping pizza, steak, sushi data...")
    zip_ref.extractall(image_path)

# Remove zip file
os.remove(data_path / "pizza_steak_sushi.zip")

data/pizza_steak_sushi directory exists.
Unzipping pizza, steak, sushi data...


Now we got these:

    data/
      pizza_steak_sushi/
        train/
          pizza/
            train_image_01.jpeg
            train_image_02.jpeg
            ...
          steak/
          sushi/
        test/
          pizza/
            test_image_01.jpeg
            test_image_02.jpeg
            ...
          steak/
          sushi/

### **2/ Create Datasets and DataLoaders**

In [10]:
### First make directory 'modular'
def create_folder(name: str='modular'):
  '''
    Create a folder named 'modular' or other names that you need
  '''
  if not os.path.isdir('modular'):
    os.mkdir('modular')
    print('Directory has been created!!!')
  else:
    print('Directory already exists')

create_folder()

Directory already exists


In [22]:
%%writefile modular/data_setup.py

"""
Contains functionality for creating PyTorch DataLoaders for
image classification data.
"""

import os
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
from typing import List, Tuple

NUM_WORKERS = os.cpu_count()

def create_dataloaders(train_dir: str=None,
                       test_dir: str=None,
                       transforms: List[transforms.Compose]=[None, None],
                       batch_size: int=None,
                       num_workers: int=NUM_WORKERS,
                       pin_memory: bool=False) -> Tuple[DataLoader, DataLoader, List[str]]:
    """Creates training and testing DataLoaders.

    Takes in a training directory and testing directory path and turns
    them into PyTorch Datasets and then into PyTorch DataLoaders.

    Args:
      train_dir: Path to training directory.
      test_dir: Path to testing directory.
      transforms: A list of two transforms: one for training and one for testing data.
      batch_size: Number of samples per batch in each of the DataLoaders.
      num_workers: An integer for number of workers per DataLoader.

    Returns:
      A tuple of (train_dataloader, test_dataloader, class_names).
      Where class_names is a list of the target classes.
    """
    # Use ImageFolder to create Datasets
    train_data = datasets.ImageFolder(train_dir, transform=transforms[0])
    test_data = datasets.ImageFolder(test_dir, transform=transforms[1])

    # Get class names
    class_names = train_data.classes

    # Turn into DataLoaders
    train_dataloader = DataLoader(train_data,
                                  batch_size=batch_size,
                                  shuffle=True,
                                  num_workers=num_workers,
                                  pin_memory=pin_memory)

    test_dataloader = DataLoader(test_data,
                                 batch_size=batch_size,
                                 shuffle=False,
                                 num_workers=num_workers,
                                 pin_memory=pin_memory)

    return train_dataloader, test_dataloader, class_names


Overwriting modular/data_setup.py


### **3/ Build Model**

In [31]:
%%writefile modular/model_builder.py

"""
Contains PyTorch model code to instantiate a TinyVGG model.
"""

import torch
import torch.nn as nn

class TinyVGG(nn.Module):
  """
    Creates the TinyVGG model

    Args:
  input_shape: An integer indicating number of input channels.
  hidden_units: An integer indicating number of hidden units between layers.
  output_shape: An integer indicating number of output units.
  """

  def __init__(self, input_shape: int, hidden_units: int, output_shape: int) -> None:
    super().__init__()
    self.conv_block_1 = nn.Sequential(
      nn.Conv2d(in_channels=input_shape,
                out_channels=hidden_units,
                kernel_size=3,
                stride=1,
                padding=0),
      nn.ReLU(),
      nn.Conv2d(in_channels=hidden_units,
                out_channels=hidden_units,
                kernel_size=3,
                stride=1,
                padding=0),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2,
                    stride=2)
    )

    self.conv_block_2 = nn.Sequential(
      nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
      nn.ReLU(),
      nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
      nn.ReLU(),
      nn.MaxPool2d(2)
    )

    self.classifier = nn.Sequential(
      nn.Flatten(),
      nn.Linear(in_features=hidden_units*13*13,
                out_features=output_shape)
    )

  def forward(self, x: torch.Tensor):
    x = self.conv_block_1(x)
    x = self.conv_block_2(x)
    x = self.classifier(x)
    return x
    # return self.classifier(self.block_2(self.block_1(x))) # <- leverage the benefits of operator fusion

Overwriting modular/model_builder.py


### **4/ Train Step and Test Step**

In [40]:
%%writefile modular/engine.py

"""
Contains functions for training and testing a PyTorch model.
"""

from typing import Dict, List, Tuple
import torch
from tqdm import tqdm

def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device: torch.device='cpu') -> Tuple[float, float]:
  """
    Trains a PyTorch model for a single epoch.

    Args:
    model: A PyTorch model to be trained.
    dataloader: A DataLoader instance for the model to be trained on.
    loss_fn: A PyTorch loss function to minimize.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    device: A target device to compute on (e.g. "cuda" or "cpu").

    Returns:
    A tuple of training loss and training accuracy metrics.
    In the form (train_loss, train_accuracy).
    For example: (0.1112, 0.8743)
  """

  # Put model into train mode
  model.train()

  # Create variables to store scores
  train_loss, train_acc = 0, 0

  # Loop through each epoch
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    y_logits = model(X)
    loss = loss_fn(y_logits, y)
    train_loss += loss.item()

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    y_pred_label = torch.argmax(torch.softmax(y_logits, dim=1), dim=1)
    train_acc += (y_pred_label == y).sum().item()/len(y_pred_label)

  # Adjust metrics to get average metrics per batch
  train_loss = train_loss / len(dataloader)
  train_acc = train_acc / len(dataloader)
  return train_loss, train_acc


def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              device: torch.device='cpu') -> Tuple[float, float]:

    """
    Tests a PyTorch model for a single epoch.

    Args:
    model: A PyTorch model to be tested.
    dataloader: A DataLoader instance for the model to be tested on.
    loss_fn: A PyTorch loss function to calculate loss on the test data.
    device: A target device to compute on (e.g. "cuda" or "cpu").

    Returns:
    A tuple of testing loss and testing accuracy metrics.
    In the form (test_loss, test_accuracy).
    For example: (0.0223, 0.8985)
    """

    # Put model into mode
    model.eval()

    # Create variables to store scores
    test_loss, test_acc = 0, 0

    # Loop through batches
    with torch.inference_mode():
      for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        test_logits = model(X)
        loss = loss_fn(test_logits, y)
        test_loss += loss.item()

        test_pred_labels = torch.argmax(torch.softmax(test_logits, dim=1), dim=1)
        test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))

    # Adjust metrics
    test_loss = test_loss / len(dataloader)
    test_acc = test_acc / len(dataloader)
    return test_loss, test_acc


def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.utils,
          num_epochs: int=5,
          device: torch.device='cpu') -> Dict[str, List[float]]:


    """
      Trains and tests a PyTorch model.

      Args:
    model: A PyTorch model to be trained and tested.
    train_dataloader: A DataLoader instance for the model to be trained on.
    test_dataloader: A DataLoader instance for the model to be tested on.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    loss_fn: A PyTorch loss function to calculate loss on both datasets.
    epochs: An integer indicating how many epochs to train for.
    device: A target device to compute on (e.g. "cuda" or "cpu").

      Returns:
    A dictionary of training and testing loss as well as training and
    testing accuracy metrics. Each metric has a value in a list for
    each epoch.
    In the form: {train_loss: [...],
              train_acc: [...],
              test_loss: [...],
              test_acc: [...]}
    For example if training for epochs=2:
             {train_loss: [2.0616, 1.0537],
              train_acc: [0.3945, 0.3945],
              test_loss: [1.2641, 1.5706],
              test_acc: [0.3400, 0.2973]}
    """

    # Create a result dictionary
    results = {"train_loss": [],
               "train_acc": [],
               "test_loss": [],
               "test_acc": []
    }

    # Loop through each epoch
    for epoch in tqdm(range(num_epochs)):
      train_loss, train_acc = train_step(model=model,
                                          dataloader=train_dataloader,
                                          loss_fn=loss_fn,
                                          optimizer=optimizer,
                                          device=device)

      test_loss, test_acc = test_step(model=model,
                                      dataloader=test_dataloader,
                                      loss_fn=loss_fn,
                                      device=device)

      # Print out what's happening
      print(
        f"Epoch: {epoch+1} | "
        f"train_loss: {train_loss:.4f} | "
        f"train_acc: {train_acc:.4f} | "
        f"test_loss: {test_loss:.4f} | "
        f"test_acc: {test_acc:.4f}"
      )

      # Update results dictionary
      results["train_loss"].append(train_loss)
      results["train_acc"].append(train_acc)
      results["test_loss"].append(test_loss)
      results["test_acc"].append(test_acc)

    # Return the filled results at the end of the epochs
    return results

Overwriting modular/engine.py


### **5/ Save Model**

In [41]:
%%writefile modular/utils.py

"""
Contains various utility functions for PyTorch model training.
"""

from pathlib import Path
import torch

def save_model(model: torch.nn.Module,
               model_name: str,
               target_dir: str='models'):

  """
    Contains various utility functions for PyTorch model training.

    Args:
      model: A PyTorch model to save.
      model_name: A name for the model file to save.
      target_dir: A directory for saving the model to.
  """

  # Create directiory
  target_dir_path = Path(target_dir)
  target_dir_path.mkdir(parents=True,
                        exist_ok=True)

  # Create model save path
  assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model_name should end with '.pt' or '.pth'"
  model_save_path = target_dir_path / model_name

  # Save the model state_dict()
  print(f"[INFO] Saving model to: {model_save_path}")
  torch.save(obj=model.state_dict(),
            f=model_save_path)


Overwriting modular/utils.py


### **6/ Start to Train, Evaluate and Save Model**

In [42]:
%%writefile modular/train_baseline.py

import torch
import os
import data_setup, engine, utils, model_builder
from torchvision import transforms

# Set hyperparameters
NUM_EPOCHS = 10
BATCH_SIZE = 32
HIDDEN_UNITS = 15
LEARNING_RATE = 0.01
NUM_WORKERS = os.cpu_count()

# Setup directories
train_dir = "data/pizza_steak_sushi/train"
test_dir = "data/pizza_steak_sushi/test"

# Setup device
device = 'cpu'
# device = "cuda" if torch.cuda.is_available() else "cpu"

# Create data transforms
train_transforms = transforms.Compose([
  transforms.Resize((64, 64)),
  transforms.TrivialAugmentWide(num_magnitude_bins=20),
  transforms.ToTensor()
])

test_transforms = transforms.Compose([
  transforms.Resize((64, 64)),
  transforms.ToTensor()
])

# Create DataLoaders
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    transforms=[train_transforms, test_transforms],
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    pin_memory=False
)


# Create model
model = model_builder.TinyVGG(input_shape=3, hidden_units=HIDDEN_UNITS, output_shape=len(class_names)).to(device)

# Create loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Start training with help from engine.py
engine.train(model=model,
             train_dataloader=train_dataloader,
             test_dataloader=test_dataloader,
             loss_fn=criterion,
             optimizer=optimizer,
             num_epochs=NUM_EPOCHS,
             device=device)

# Save the model with help from utils.py
utils.save_model(model=model,
                 target_dir="models",
                 model_name="tinyvgg_model.pth")

Overwriting modular/train_baseline.py


In [43]:
!python modular/train_baseline.py

  0% 0/10 [00:00<?, ?it/s]Epoch: 1 | train_loss: 1.1531 | train_acc: 0.2734 | test_loss: 1.0939 | test_acc: 0.5417
 10% 1/10 [00:02<00:19,  2.21s/it]Epoch: 2 | train_loss: 1.0951 | train_acc: 0.4297 | test_loss: 1.0951 | test_acc: 0.2604
 20% 2/10 [00:04<00:16,  2.09s/it]Epoch: 3 | train_loss: 1.1047 | train_acc: 0.3047 | test_loss: 1.1009 | test_acc: 0.2604
 30% 3/10 [00:06<00:14,  2.03s/it]Epoch: 4 | train_loss: 1.0910 | train_acc: 0.4258 | test_loss: 1.1079 | test_acc: 0.2604
 40% 4/10 [00:09<00:15,  2.56s/it]Epoch: 5 | train_loss: 1.1049 | train_acc: 0.3047 | test_loss: 1.1123 | test_acc: 0.2604
 50% 5/10 [00:11<00:11,  2.37s/it]Epoch: 6 | train_loss: 1.1035 | train_acc: 0.3047 | test_loss: 1.1073 | test_acc: 0.2604
 60% 6/10 [00:13<00:08,  2.25s/it]Epoch: 7 | train_loss: 1.1002 | train_acc: 0.3047 | test_loss: 1.0991 | test_acc: 0.2604
 70% 7/10 [00:15<00:06,  2.17s/it]Epoch: 8 | train_loss: 1.1061 | train_acc: 0.3047 | test_loss: 1.0917 | test_acc: 0.2604
 80% 8/10 [00:17<00:04, 

### **7/ Load Saved Model**

In [47]:
import torch
import torchinfo
from modular import model_builder

saved_model = model_builder.TinyVGG(input_shape=3, hidden_units=15, output_shape=3).to('cpu')
saved_model.load_state_dict(torch.load('models/tinyvgg_model.pth'))
torchinfo.summary(saved_model, (1, 3, 64, 64))

ModuleNotFoundError: No module named 'torchinfo'